# Detection of Bruxism events in Tinnitus patients polysomnographic data
This notebook will 
- load EMG channels of polusomnographic data
- detect the EMG bursts in a unsupervised way
- classify EMG bursts as different bruxism events
- give insights on the bruxism events

In [1]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import mne
import scipy
import seaborn as sns
from tinnsleep.config import Config
from tinnsleep.data import CreateRaw, RawToEpochs_sliding, CleanAnnotations, AnnotateRaw_sliding
from tinnsleep.classification import AmplitudeThresholding
from tinnsleep.check_impedance import create_annotation_mne, Impedance_thresholding_sliding, check_RMS, fuse_with_classif_result
from tinnsleep.signal import rms
from tinnsleep.visualization import plotTimeSeries, plotAnnotations, zoom_effect
from tinnsleep.create_reports import create_reports
from IPython.core.display import display
from ipywidgets import widgets
print("Config loaded")


Config loaded


## Load, filter, and prepare data

In [41]:
filename = Config.bruxisme_files[0]  # load file from config
picks_chan = ['1', '2']           # subset of EMG electrodes

raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading
tmin = raw.times[0]                     
tmax = raw.times[-1]

croptimes=dict(tmin=raw.times[0]+3600*2, tmax=raw.times[-1]-3600)
raw.crop(**croptimes)

raw  = CreateRaw(raw[picks_chan][0], picks_chan, ch_types=['emg'])        # pick channels and load

raw  = raw.filter(20., 99., n_jobs=4, 
                  fir_design='firwin', filter_length='auto', phase='zero-double',
                  picks=picks_chan)
ch_names = raw.info["ch_names"]
print("Data filtered")

offset = raw.times[0]
print(f"keeping {(raw.times[-1]-raw.times[0])/3600:0.2f} hours of recording out of {(tmax-tmin)/3600:0.2f} hours")

Extracting EDF parameters from C:\Users\zeta\documents\EEG_polysomno\PSG2\1BA07_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-41-5db0a3ad1304>:4: RuntimeWarning: 7 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Intensit? lumine', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


Data filtered
keeping 7.28 hours of recording out of 8.83 hours


## Epoching data

In [42]:
sfreq = raw.info["sfreq"]
window_length = 0.25                    # in seconds
duration = int(window_length * sfreq)   # in samples
interval = duration                     # no overlapping
epochs = RawToEpochs_sliding(raw, duration=duration, interval=interval)
print(f"Epochs done, shape {epochs.shape}")


Epochs done, shape (104900, 2, 50)


## Get the impedance & artefacts annotations

In [43]:
# Value of the impedance threshold
THR_imp = 5000

raw_imp  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading
ch_names = raw_imp.info["ch_names"]
picks_chan = [ch_names[1],ch_names[5]]
print(picks_chan)

croptimes=dict(tmin=raw_imp.times[0]+3600*2, tmax=raw_imp.times[-1]-3600)
raw_imp.crop(**croptimes)

#Get the table of bad electrodes booleans from the impedance thresholding algo
check_imp  = Impedance_thresholding_sliding(raw_imp[picks_chan][0], duration, interval,THR_imp) 
print(check_imp[:3])

# convert to labels per epoch
impedance_labels = np.any(check_imp, axis=-1)
print(f"rejected impedances for {np.sum(impedance_labels)} epochs out of {len(impedance_labels)} ({np.sum(impedance_labels)/len(impedance_labels)*100:.2f}%)")

Extracting EDF parameters from C:\Users\zeta\documents\EEG_polysomno\PSG2\1BA07_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['1 Imp?dance', '2 Imp?dance']


<ipython-input-43-97476cdf5950>:4: RuntimeWarning: 7 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Intensit? lumine', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw_imp  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


[[False False]
 [False False]
 [False False]]
rejected impedances for 26 epochs out of 104900 (0.02%)


In [44]:
# baseline epochs amplitudes
print(f"mean {np.mean(np.mean(epochs, axis=0), axis=-1)}")
print(f"median {np.median(np.median(epochs, axis=0), axis=-1)}")
print(f"quantile 0.05 {np.quantile(np.quantile(epochs, 0.05, axis=0),0.05, axis=-1)}")
print(f"quantile 0.95 {np.quantile(np.quantile(epochs, 0.95, axis=0),0.95, axis=-1)}")

mean [-3.18235151e-13 -1.89347516e-12]
median [2.82419815e-08 1.01926332e-08]
quantile 0.05 [-7.01076668e-06 -7.10918958e-06]
quantile 0.95 [6.95685044e-06 7.09699822e-06]


In [45]:
# Epoch rejection based on |min-max| thresholding 
from tinnsleep.signal import is_good_epochs
params = dict(ch_names=raw.info["ch_names"],
             rejection_thresholds=dict(emg=1e-04), # two order of magnitude higher q0.01
             flat_thresholds=dict(emg=1e-09),    # one order of magnitude lower median
             channel_type_idx=dict(emg=[0, 1]),
             full_report=True
            )
amplitude_labels, bad_lists = is_good_epochs(epochs, **params)
print(amplitude_labels[:10])
print(bad_lists[:10])
print(f"good amplitudes for {np.sum(amplitude_labels)} epochs out of {len(amplitude_labels)} ({np.sum(amplitude_labels)/len(amplitude_labels)*100:.2f}%)")

[True, True, True, True, True, True, True, True, True, True]
[None, None, None, None, None, None, None, None, None, None]
good amplitudes for 102344 epochs out of 104900 (97.56%)


In [46]:
# Merge is_good and amplitude
#TODO: THERE IS A MISSMATCH BETWEEN LABEL SIZE
valid_labels = np.any(np.c_[np.invert(impedance_labels),  amplitude_labels], axis=-1) # Logical OR
print(valid_labels[:3])

dict_annotations_artefacts = {1: "artefact"}
annotations_artefacts = []
for k, label in enumerate(np.invert(valid_labels)):
    if label > 0:
        annotations_artefacts.append(dict(
            onset=k*interval/sfreq,
            duration=duration/sfreq,
            description=dict_annotations_artefacts[label],
            orig_time=offset
        )
            
        )

[ True  True  True]


## Classifying epochs and annotate raw

In [47]:
# compute the sum of power over electrodes and samples in each window
pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=2)
X        = rms(epochs[valid_labels]) # take only valid labels
labels   = pipeline.fit_predict(X)
labels   = fuse_with_classif_result(np.invert(valid_labels), labels) # add the missing labels removed with artefacts
print(f"bursts count: {np.sum(labels)}/{len(labels)} ({np.sum(labels) / len(labels) * 100:.2f}%)")
print(f"bursts time: {np.sum(labels) * window_length} seconds")

dict_annotations = {1: "burst"}
annotations = []
for k, label in enumerate(labels):
    if label > 0:
        annotations.append(dict(
            onset=k*interval/sfreq,
            duration=duration/sfreq,
            description=dict_annotations[label],
            orig_time=offset
        )
            
        )

bursts count: 6689/104900 (6.38%)
bursts time: 1672.25 seconds


## Display Annotations

In [48]:
plt.close("all")

# decimate signal to make it more readible 
raw_ds = raw.copy().resample(100)

%matplotlib widget
scalings=1e-4

ax1 = plt.subplot(211)
plotTimeSeries(raw_ds.get_data().T, sfreq=raw_ds.info["sfreq"], ax=ax1, scalings=scalings, offset=offset)
plotAnnotations(annotations, color="red")
plotAnnotations(annotations_artefacts, color="green")

ax1.set_xlim(5145,5165)
ax2 = plt.subplot(212)
plotTimeSeries(raw_ds.get_data().T, sfreq=raw_ds.info["sfreq"], ax=ax2, scalings=scalings, offset=offset)
z = zoom_effect(ax1, ax2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# update manually the time axis
ax1.set_xlim(5145,5165) # in seconds

(5145, 5165)

### (OPTIONAL) Enable widget

In [16]:
from ipywidgets import interact, FloatSlider
def update_axis(xmin, xmax):
    if xmin<xmax:
        ax1.set_xlim(xmin,xmax)

i=FloatSlider(min=raw.times[0], max=raw.times[-1], step=10, continuous_update=False)
ii=FloatSlider(min=raw.times[0], max=raw.times[-1], step=10, continuous_update=False)
from ipywidgets import FloatSlider
interact(update_axis,xmin=i, xmax=ii);

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='xmin', max=26224.995, step=…

## Create reports for several recordings

In [3]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import mne
from tinnsleep.config import Config
from tinnsleep.create_reports import create_reports

#Redefining parameters
EDF_list = Config.bruxisme_files[0:2]
ind_picks_chan = [0, 4]
ind_picks_imp = [1, 5]
THR_classif=[2, 3, 4, 5, 6, 7, 8, 9, 10]
sfreq = 200
window_length = 0.25                    # in seconds
duration = int(window_length * sfreq)   # in samples
interval = duration                     # no overlapping

results = create_reports(EDF_list, ind_picks_chan, ind_picks_imp, THR_classif, duration, interval, THR_imp = 8000)

Extracting EDF parameters from C:\Users\zeta\documents\EEG_polysomno\PSG2\1BA07_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Zeta\Documents\acou_sommeil_HD_ENS\Tinnitus-n-Sleep\Notebooks/..\tinnsleep\create_reports.py:47: RuntimeWarning: 7 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Intensit? lumine', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


Extracting EDF parameters from C:\Users\zeta\documents\EEG_polysomno\PSG2\1BA07_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Zeta\Documents\acou_sommeil_HD_ENS\Tinnitus-n-Sleep\Notebooks/..\tinnsleep\create_reports.py:77: RuntimeWarning: 7 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Intensit? lumine', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw_imp = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


Extracting EDF parameters from C:\Users\zeta\documents\EEG_polysomno\PSG2\1DA15_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Zeta\Documents\acou_sommeil_HD_ENS\Tinnitus-n-Sleep\Notebooks/..\tinnsleep\create_reports.py:47: RuntimeWarning: 6 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


Extracting EDF parameters from C:\Users\zeta\documents\EEG_polysomno\PSG2\1DA15_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Zeta\Documents\acou_sommeil_HD_ENS\Tinnitus-n-Sleep\Notebooks/..\tinnsleep\create_reports.py:77: RuntimeWarning: 6 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw_imp = mne.io.read_raw_edf(filename, preload=False)  # prepare loading
C:\Users\Zeta\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Zeta\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Zeta\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Zeta\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Zeta\Ana

In [10]:
print(results.keys())
print(results['C:\\Users\\zeta\\documents\\EEG_polysomno\\PSG2\\1BA07_nuit_hab.edf']["THR_classif"][0])
print(results['C:\\Users\\zeta\\documents\\EEG_polysomno\\PSG2\\1BA07_nuit_hab.edf']["reports"][0])

dict_keys(['C:\\Users\\zeta\\documents\\EEG_polysomno\\PSG2\\1BA07_nuit_hab.edf', 'C:\\Users\\zeta\\documents\\EEG_polysomno\\PSG2\\1DA15_nuit_hab.edf'])
2
{'Recording duration': 32035.25, 'Total burst duration': 2417.25, 'Total number of burst': 2156, 'Number of bursts per hour': 242.28311001162783, 'Total number of episodes': 228, 'Number of bursts per episode': 9.456140350877194, 'Number of episodes per hour': 25.621776012361384, 'Number of tonic episodes per hour': 6.630196424251411, 'Number of phasic episodes per hour': 2.1632420536752486, 'Number of mixed episodes per hour': 2.5846528433522447, 'Mean duration of tonic episode': 4.110169491525424, 'Mean duration of phasic episode': 7.142857142857143, 'Mean duration of mixed episode': 27.391304347826086}
